## Import modules

In [1]:
from System.paths import *
from OMM_interface.openmm import *
import os, re
import numpy as np
from pathlib import Path
from ASE_interface.ase_calculation import CP2K
from Coord_Toolz.mdanalysis import MDA_reader
from Direct_cp2k_calculation.direct_cp2k import Direct_Calculator
from System.paths import Paths
from System.system import *
from ASE_interface.ase_calculation import ASE_system
from Parametrization.parametrization import Parametrization
from Parametrization.optimizers import Optimizer
from Coord_Toolz.mdanalysis import *
from OMM_interface.openmm import OpenMM_system

## Configure Paths

In [2]:
system_name = 'ep2'

In [3]:
paths = Paths()

paths.working_dir = '/home/ac127777/Documents/TheForceOutput/'+system_name
paths.project_name = system_name

paths.mm_top_file = 'smallbox.psf'
paths.mm_top_file_path = '/home/ac127777/Documents/phosphate_parametrization/cp2k_geom_opts/'+system_name
paths.mm_crd_file = 'structure_0.pdb'
paths.mm_crd_file_path = paths.mm_top_file_path
paths.mm_str_file = 'ep2'+'.str'
paths.mm_str_file_path = '/home/ac127777/Documents/phosphate_parametrization/cp2k_geom_opts/'+system_name
paths.mm_traj_file = 'smallbox.dcd'
paths.mm_traj_file_path = '/home/ac127777/Documents/phosphate_parametrization/cp2k_geom_opts/'+system_name

paths.set()

Created folder /home/ac127777/Documents/TheForceOutput/ep2


## Import Molecules:

Set up System.system.Molecular_system

In [4]:
Molsys = Molecular_system('net_properties', 'forces')

In [5]:
Molsys.paths = paths

Set up Coord_Toolz.mdanalysis.MDA_reader object for both molecules dissolved, one molecule dissolved, and the other molecule dissolved

In [6]:
MDR = MDA_reader()
MDR.set_traj_input(paths.mm_crd, paths.mm_traj)

/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"


In [7]:
MDR.universes['mol1'] = MDR.delete_one_molecule('not resname SOD') #ep2
MDR.universes['mol2'] = MDR.delete_one_molecule('not resname EP2')

export nosol

export mol1 and mol2

In [8]:
write_single_pdb(MDR.universes['mol1'], 'mol1.pdb', paths.working_dir)
write_single_pdb(MDR.universes['mol2'], 'mol2.pdb', paths.working_dir)

/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/coordinates/PDB.py:1129: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


...externally generate topology files for mol1 and mol2....
e.g. run 'charmm -i genpsf.inp -o genpsf.out'

add paths to coordinate and topology files

In [9]:
paths.working_dir

'/home/ac127777/Documents/TheForceOutput/ep2/'

In [10]:
paths.mm_mol1_crd_file_path = paths.working_dir
paths.mm_mol1_crd_file = 'mol1.pdb'

paths.mm_mol1_top_file_path = paths.working_dir
paths.mm_mol1_top_file = 'mol1.psf'

paths.mm_mol2_crd_file_path = paths.working_dir
paths.mm_mol2_crd_file = 'mol2.pdb'

paths.mm_mol2_top_file_path = paths.working_dir
paths.mm_mol2_top_file = 'mol2.psf'

paths.set()

check if they work

In [11]:
for filepath in [paths.mm_mol1_crd, paths.mm_mol1_top, paths.mm_mol2_crd, paths.mm_mol2_top]:
    check_if_file_exists(filepath)

Fill Molecular_system storage

In [12]:
Molsys.set_ini_coords(MDR)

## Run QM scheme

### option 1.1: no charge calc, no optimization

define cp2k_inp

Run the DFT calculation

### option 1.2: no charge calc, geometry optimization

define cp2k_inp

Run the DFT calculation

use the new optimized coords and run a single-point calculation:

generate a new MDA_reader object

exchange the ini_coords for the opt_coords

Run the DFT calculation

### option 2.1: calculate charges, energies, forces, no geom opt

define cp2k_inp

In [13]:
cp2k_inp2 = """  
&FORCE_EVAL
  METHOD Quickstep              ! Electronic structure method (DFT,...)
  &DFT
    BASIS_SET_FILE_NAME  BASIS_MOLOPT_UZH
    POTENTIAL_FILE_NAME  POTENTIAL_UZH
    CHARGE 0
    &MGRID
      NGRIDS 5
      CUTOFF 550
      REL_CUTOFF 80
    &END MGRID
    &QS
      METHOD GPW
      EPS_DEFAULT 1.0E-12
    &END QS
    &POISSON                    ! Solver requested for non periodic calculations
      PERIODIC XYZ
      PSOLVER  PERIODIC          ! Type of solver
    &END POISSON
    &SCF                        ! Parameters controlling the convergence of the scf. This section should not be changed. 
      SCF_GUESS ATOMIC
      EPS_SCF 1.0E-5
      MAX_SCF 800
      &MIXING
        ALPHA 0.4
      &END MIXING
      &OT
        MINIMIZER CG
      &END OT
    &END SCF
    &XC                        ! Parameters needed to compute the electronic exchange potential 
      &VDW_POTENTIAL
        DISPERSION_FUNCTIONAL PAIR_POTENTIAL
        &PAIR_POTENTIAL
          TYPE DFTD3
          PARAMETER_FILE_NAME  dftd3.dat
          REFERENCE_FUNCTIONAL PBE
          CALCULATE_C9_TERM TRUE
          REFERENCE_C9_TERM TRUE
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_FUNCTIONAL PBE
      &END XC_FUNCTIONAL
    &END XC
  &END DFT

  &SUBSYS
    &KIND H
      ELEMENT H
      BASIS_SET TZV2P-MOLOPT-PBE-GTH-q1
      POTENTIAL GTH-PBE-q1
    &END KIND
    &KIND C
      ELEMENT C
      BASIS_SET TZV2P-MOLOPT-PBE-GTH-q4
      POTENTIAL GTH-PBE-q4
    &END KIND
    &KIND P
      ELEMENT P
      BASIS_SET TZV2P-MOLOPT-PBE-GTH-q5
      POTENTIAL GTH-PBE-q5
    &END KIND
    &KIND O
      ELEMENT O
      BASIS_SET TZV2P-MOLOPT-PBE-GTH-q6
      POTENTIAL GTH-PBE-q6
    &END KIND
    &KIND NA
      ELEMENT Na
      BASIS_SET TZV2P-MOLOPT-PBE-GTH-q9
      POTENTIAL GTH-PBE-q9
    &END KIND
    &KIND N
      ELEMENT N
      BASIS_SET TZV2P-MOLOPT-PBE-GTH-q5
      POTENTIAL GTH-PBE-q5
    &END KIND
    &PRINT
      &ATOMIC_COORDINATES LOW
      &END ATOMIC_COORDINATES
      &INTERATOMIC_DISTANCES LOW
      &END INTERATOMIC_DISTANCES
      &KINDS 
        POTENTIAL
      &END KINDS
      &TOPOLOGY_INFO
        PSF_INFO
      &END TOPOLOGY_INFO
    &END PRINT
  &END SUBSYS

  &PROPERTIES
    &RESP
      &SPHERE_SAMPLING
            AUTO_VDW_RADII_TABLE CAMBRIDGE
            AUTO_RMIN_SCALE 1.0
            AUTO_RMAX_SCALE 3.0
      &END
      &PRINT
        &COORD_FIT_POINTS
        &END
        &V_RESP_CUBE
        &END
      &END
    &END RESP
  &END PROPERTIES

  &PRINT
    &FORCES ON
    &END FORCES
  &END PRINT

&END FORCE_EVAL """

Run the DFT calculation

### option 2.2: calculate charges, energies, forces, and optimize geometry

define cp2k_inp

Run DFT calc

use the new optimized coords and run a single-point calculation:

generate a new MDA_reader object

exchange the ini_coords for the opt_coords

Run the DFT calculation

### option 3.1: read in data from external cluster calcs

use the new optimized coords and run a single-point calculation:

generate a new MDA_reader object

exchange the ini_coords for the opt_coords

Run the DFT calculation

### option 3.2: read in all data from external cluster calcs

entire system

In [13]:
Molsys.read_qm_charges_energies_forces_optcoords('all', paths.mm_top_file_path, 'geom_opt', 'ep2'+'_opt', 'RESP') #ep2

reading qm frame 0 info


/home/ac127777/anaconda3/envs/mdanalysis-dev/lib/python3.8/site-packages/MDAnalysis/analysis/base.py:438: UserWarning: Reader has no dt information, set to 1.0 ps
  self.times[i] = ts.time


reading qm frame 1 info
reading qm frame 2 info
reading qm frame 3 info
reading qm frame 4 info
reading qm frame 5 info
reading qm frame 6 info
reading qm frame 7 info
reading qm frame 8 info
reading qm frame 9 info
reading qm frame 10 info
reading qm frame 11 info
reading qm frame 12 info
reading qm frame 13 info
reading qm frame 14 info
reading qm frame 15 info
reading qm frame 16 info
reading qm frame 17 info
reading qm frame 18 info
reading qm frame 19 info
reading qm frame 20 info
reading qm frame 21 info
reading qm frame 22 info
reading qm frame 23 info
reading qm frame 24 info
reading qm frame 25 info
reading qm frame 26 info
reading qm frame 27 info
reading qm frame 28 info
reading qm frame 29 info
reading qm frame 30 info
reading qm frame 31 info
reading qm frame 32 info
reading qm frame 33 info
reading qm frame 34 info
reading qm frame 35 info
reading qm frame 36 info
reading qm frame 37 info
reading qm frame 38 info
reading qm frame 39 info
reading qm frame 40 info
reading q

net forces: nosol or mol1&mol2

In [14]:
for system_type in ['mol1', 'mol2']:
    print(system_type)
    Molsys.read_qm_charges_energies_forces_optcoords(system_type, paths.mm_stream[:-8]+'_sp/', system_type+'_sp', 'cp2k_direct') #ep2

mol1
reading qm frame 0 info
reading qm frame 1 info
reading qm frame 2 info
reading qm frame 3 info
reading qm frame 4 info
reading qm frame 5 info
reading qm frame 6 info
reading qm frame 7 info
reading qm frame 8 info
reading qm frame 9 info
reading qm frame 10 info
reading qm frame 11 info
reading qm frame 12 info
reading qm frame 13 info
reading qm frame 14 info
reading qm frame 15 info
reading qm frame 16 info
reading qm frame 17 info
reading qm frame 18 info
reading qm frame 19 info
reading qm frame 20 info
reading qm frame 21 info
reading qm frame 22 info
reading qm frame 23 info
reading qm frame 24 info
reading qm frame 25 info
reading qm frame 26 info
reading qm frame 27 info
reading qm frame 28 info
reading qm frame 29 info
reading qm frame 30 info
reading qm frame 31 info
reading qm frame 32 info
reading qm frame 33 info
reading qm frame 34 info
reading qm frame 35 info
reading qm frame 36 info
reading qm frame 37 info
reading qm frame 38 info
reading qm frame 39 info
readi

generate a new MDA_reader object

In [15]:
MDR_optcoords = MDA_reader()

load the opt_coords - 'all' system

In [16]:
MDR_optcoords.universes['all'] = merge_atoms_positions(Molsys.opt_coords['all'], MDR.universes['all'].atoms)

load the opt_coords - nosol/mol1/mol2

In [17]:
MDR_optcoords.universes['mol1'] = MDR_optcoords.delete_one_molecule('not resname SOD') #ep2
MDR_optcoords.universes['mol2'] = MDR_optcoords.delete_one_molecule('not resname EP2')

optional but recommended: Set ini_coords to optcoords

In [18]:
Molsys.set_ini_coords(MDR_optcoords) 

### calculate QM net forces

In [19]:
Molsys.calculate_qm_net_forces()

## Run MM scheme

Set up OpenMM system(s) and link them to Molecular_system

first do entire system

In [20]:
omm_sys_all = OpenMM_system(topology_format='CHARMM', top_file=paths.mm_top, crd_format='PDB',
                        crd_file=paths.mm_crd, charmm_param_file=paths.mm_stream, pbc=True,
                        cell=[1.6, 1.6, 1.6], angles=[90, 90, 90])

In [21]:
omm_sys_all.create_system_params['nonbondedCutoff'] = 0.8 * unit.nanometer
omm_sys_all.create_system_params['constraints'] = app.HBonds
omm_sys_all.import_molecular_system()

/home/ac127777/Documents/TheForce/OMM_interface
/home/ac127777/Documents/TheForce/OMM_interface/top_all36_cgenff.rtf


In [22]:
omm_sys_all.set_integrator()


In [23]:
omm_sys_all.set_platform()
omm_sys_all.create_openmm_system()

In [24]:
omm_sys_all.set_openmm_context()

Number of constraints: 237
constraints settings: HBonds
make sure these are correct before setting context


In [25]:
Molsys.openmm_systems['all'] = omm_sys_all

then for molecule 1 

In [26]:
omm_sys_mol1 = OpenMM_system(topology_format='CHARMM', top_file=paths.mm_mol1_top, crd_format='PDB',
                        crd_file=paths.mm_mol1_crd, charmm_param_file=paths.mm_stream, pbc=True,
                        cell=[1.6, 1.6, 1.6], angles=[90, 90, 90])

In [27]:
omm_sys_mol1.create_system_params['nonbondedCutoff'] = 0.8 * unit.nanometer
omm_sys_mol1.create_system_params['constraints'] = app.HBonds
omm_sys_mol1.import_molecular_system()
omm_sys_mol1.set_integrator()
omm_sys_mol1.set_platform()
omm_sys_mol1.create_openmm_system()
omm_sys_mol1.set_openmm_context()

/home/ac127777/Documents/TheForce/OMM_interface
/home/ac127777/Documents/TheForce/OMM_interface/top_all36_cgenff.rtf
Number of constraints: 237
constraints settings: HBonds
make sure these are correct before setting context


In [28]:
Molsys.openmm_systems['mol1'] = omm_sys_mol1

...and molecule 2

In [29]:
omm_sys_mol2 = OpenMM_system(topology_format='CHARMM', top_file=paths.mm_mol2_top, crd_format='PDB',
                        crd_file=paths.mm_mol2_crd, charmm_param_file=paths.mm_stream, pbc=True,
                        cell=[1.6, 1.6, 1.6], angles=[90, 90, 90])

In [30]:
omm_sys_mol2.create_system_params['nonbondedCutoff'] = 0.8 * unit.nanometer
omm_sys_mol2.create_system_params['constraints'] = app.HBonds
omm_sys_mol2.import_molecular_system()
omm_sys_mol2.set_integrator()
omm_sys_mol2.set_platform()
omm_sys_mol2.create_openmm_system()
omm_sys_mol2.set_openmm_context()

/home/ac127777/Documents/TheForce/OMM_interface
/home/ac127777/Documents/TheForce/OMM_interface/top_all36_cgenff.rtf
Number of constraints: 232
constraints settings: HBonds
make sure these are correct before setting context


In [31]:
Molsys.openmm_systems['mol2'] = omm_sys_mol2

...or the molecule without solvent

### ...now extract the force field... 

In [32]:
for omm_system_name in Molsys.openmm_systems.keys():
    if Molsys.openmm_systems[omm_system_name] != None:
        Molsys.openmm_systems[omm_system_name].extract_forcefield()

In [33]:
for omm_system_name in Molsys.openmm_systems.keys():
    if Molsys.openmm_systems[omm_system_name] != None:
        Molsys.openmm_systems[omm_system_name].create_force_field_optimizable(opt_charges=True, opt_lj=True)

In [34]:
for omm_system_name in Molsys.openmm_systems.keys():
    if Molsys.openmm_systems[omm_system_name] != None:
        Molsys.get_mm_charges(omm_system_name)

### ...calculate the classical forces...

In [35]:
for omm_system_name in Molsys.openmm_systems.keys():
    if Molsys.openmm_systems[omm_system_name] != None:
        Molsys.generate_mm_energies_forces(omm_system_name, verbose = True)

########################################
# calculated MM F&E of 50 frames
########################################
########################################
# calculated MM F&E of 50 frames
########################################
########################################
# calculated MM F&E of 50 frames
########################################


### calculate MM net forces

In [36]:
Molsys.calculate_mm_net_forces()

In [37]:
mm_forces_ini = copy.deepcopy(Molsys.mm_forces)
mm_net_forces_ini = copy.deepcopy(Molsys.mm_net_forces)

### reduce parameter set

In [38]:
Molsys.get_types_from_psf_topology_file()

{'nosol': [],
 'all': array([[0, 'CG321'],
        [1, 'OG303'],
        [2, 'PG2'],
        [3, 'OG2P1'],
        [4, 'OG2P1'],
        [5, 'OG2P1'],
        [6, 'HGA2'],
        [7, 'HGA2'],
        [8, 'CG331'],
        [9, 'HGA3'],
        [10, 'HGA3'],
        [11, 'HGA3'],
        [12, 'OT'],
        [13, 'HT'],
        [14, 'HT'],
        [15, 'OT'],
        [16, 'HT'],
        [17, 'HT'],
        [18, 'OT'],
        [19, 'HT'],
        [20, 'HT'],
        [21, 'OT'],
        [22, 'HT'],
        [23, 'HT'],
        [24, 'OT'],
        [25, 'HT'],
        [26, 'HT'],
        [27, 'OT'],
        [28, 'HT'],
        [29, 'HT'],
        [30, 'OT'],
        [31, 'HT'],
        [32, 'HT'],
        [33, 'OT'],
        [34, 'HT'],
        [35, 'HT'],
        [36, 'OT'],
        [37, 'HT'],
        [38, 'HT'],
        [39, 'OT'],
        [40, 'HT'],
        [41, 'HT'],
        [42, 'OT'],
        [43, 'HT'],
        [44, 'HT'],
        [45, 'OT'],
        [46, 'HT'],
        [47, 'HT'],


In [39]:
slice_list = {'all': [np.r_[1:6]], #ep2
              'mol1': [np.r_[1:6]],
              }

by defining the atoms for which the parameters are supposed to be optimized in the slice_list

and applying it, automatically removing duplicates

In [40]:
Molsys.reduce_ff_optimizable(slice_list)

hybrid check not performed
performing hybrid check...
hybrid sys found
hybrid set to True


In [41]:
Molsys.reduced_indexed_ff_optimizable

{'all': {'CustomNonbondedForce': [array([[0.293996576986312, 0.41840000000000005, 'OG303', 1],
          [0.38308644880034587, 2.44764, 'PG2', 2],
          [0.30290556416771536, 0.50208, 'OG2P1', 3]], dtype=object),
   array([[0.31600000000000006, 0.31388368000000005, 'SOD', 'OG2P1', 1]],
         dtype=object)],
  'NonbondedForce': [array([[1, 'OG303', -0.401],
          [2, 'PG2', 1.1],
          [3, 'OG2P1', -0.9]], dtype=object)]},
 'mol1': {'NonbondedForce': [array([[1, 'OG303', -0.401, 0.29399657698631193, 0.41840000000000005],
          [2, 'PG2', 1.1, 0.38308644880034587, 2.44764],
          [3, 'OG2P1', -0.9, 0.30290556416771536, 0.50208]], dtype=object)]}}

In [42]:
Molsys.reduced_ff_optimizable_values

{'all': {'CustomNonbondedForce': [array([[0.293996576986312, 0.41840000000000005],
          [0.38308644880034587, 2.44764],
          [0.30290556416771536, 0.50208]], dtype=object),
   array([[0.31600000000000006, 0.31388368000000005]], dtype=object)],
  'NonbondedForce': [array([[-0.401],
          [1.1],
          [-0.9]], dtype=object)]},
 'mol1': {'NonbondedForce': [array([[-0.401, 0.29399657698631193, 0.41840000000000005],
          [1.1, 0.38308644880034587, 2.44764],
          [-0.9, 0.30290556416771536, 0.50208]], dtype=object)]}}

vectorize 'em

In [43]:
Molsys.vectorize_reduced_parameters()

In [44]:
Molsys.vectorized_reduced_ff_optimizable_values

{'CustomNonbondedForce': [array([0.293996576986312, 0.41840000000000005, 0.38308644880034587,
         2.44764, 0.30290556416771536, 0.50208], dtype=object),
  array([0.31600000000000006, 0.31388368000000005], dtype=object)],
 'NonbondedForce': [array([-0.401, 1.1, -0.9], dtype=object)]}

In [45]:
Molsys.merge_vectorized_parameters()

In [46]:
Molsys.vectorized_parameters

array([ 0.29399658,  0.4184    ,  0.38308645,  2.44764   ,  0.30290556,
        0.50208   ,  0.316     ,  0.31388368, -0.401     ,  1.1       ,
       -0.9       ])

scale 'em

In [47]:
Molsys.scale_initial_parameters()

In [48]:
Molsys.scaled_parameters

array([-0.05152249, -0.00829654, -0.02449725,  6.1566378 , -0.04971239,
        0.04253306, -0.04669195, -0.04720914,  0.41845158,  0.91487634,
        1.50023674])

In [49]:
Molsys.scaling_factors # have to be const

array([-0.1752486 , -0.0198292 , -0.06394705,  2.51533632, -0.16411845,
        0.08471371, -0.14775934, -0.15040329, -1.04352016,  0.83170577,
       -1.66692971])

add weights

In [50]:
# throw out faulty frame16 from ep2
weights = np.ones((Molsys.n_conformations))
weights[16] = 0
weights = weights / (np.sum(weights))
Molsys.weights = weights

In [51]:
weights

array([0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.        , 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333,
       0.02083333, 0.02083333, 0.02083333, 0.02083333, 0.02083333])

### optimize

initialize Optimizer and change settings

In [52]:
opt_dict = {'adaptive': True} # Nelder-mead only

In [53]:
Opti = Optimizer('scipy_local', {'method': 'Nelder-Mead', 'options': opt_dict})
Opti.enforce_constraints = True

In [54]:
Opti.max_iterations = 10000

In [55]:
Opti.tolerance = 1e-5

In [56]:
Opti.opt_settings

{'method': 'Nelder-Mead', 'options': {'adaptive': True}}

initialize Parametrization module

In [57]:
Para = Parametrization(Molsys, 'force_c', Opti, 'auto')

Run the parametrization loop

In [58]:
Para.parametrize(Molsys.scaled_parameters)

###################################################################################
#                         STARTING SCIPY_LOCAL OPTIMIZER                            
###################################################################################
###################################################################################
#                         SCIPY_LOCAL OPTIMIZER FINISHED                            
###################################################################################
optimized parameters:
[-0.05667986 -0.00840798 -0.01379741  6.18760802 -0.05311327  0.04423202
 -0.04961809 -0.05185146  0.4448418   0.92703201  1.5174607 ]
objective function value:
0.01356179044516525


In [59]:
Opti.opt_settings

{'method': 'Nelder-Mead',
 'options': {'maxiter': 10000},
 'tol': 1e-05,
 'bounds': array([[-1.75248605e-01, -5.25745814e-03],
        [-3.96584035e-01, -3.96584035e-05],
        [-6.39470476e-02, -1.91841143e-03],
        [ 5.03067264e-03,  5.03067264e+01],
        [-1.64118449e-01, -4.92355347e-03],
        [ 1.69427413e-04,  1.69427413e+00],
        [-1.47759338e-01, -2.95518676e-02],
        [-2.33125106e+00, -9.02419765e-03],
        [-0.00000000e+00,  2.08704031e+00],
        [ 0.00000000e+00,  4.99023460e+00],
        [-0.00000000e+00,  3.33385941e+00]])}

check output

In [60]:
Para.parameters # params out

array([-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
        0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.92703201,
        1.5174607 ])

In [61]:
Para.iterations

25743

In [65]:
Molsys.reduced_indexed_ff_optimizable

{'all': {'CustomNonbondedForce': [array([[0.3234254759161587, 0.4240201476359742, 'OG303', 1],
          [0.2157630343060931, 2.459952558318135, 'PG2', 2],
          [0.32362767388845814, 0.5221353272207353, 'OG2P1', 3]],
         dtype=object),
   array([[0.33580341155994586, 0.3447495018373475, 'SOD', 'OG2P1', 1]],
         dtype=object)],
  'NonbondedForce': [array([[1, 'OG303', -0.4262896137364945],
          [2, 'PG2', 1.1146153461471149],
          [3, 'OG2P1', -0.9103327452445908]], dtype=object)]},
 'mol1': {'NonbondedForce': [array([[1, 'OG303', -0.4262896137364945, 0.3234254759161587,
           0.4240201476359742],
          [2, 'PG2', 1.1146153461471149, 0.2157630343060931,
           2.459952558318135],
          [3, 'OG2P1', -0.9103327452445908, 0.32362767388845814,
           0.5221353272207353]], dtype=object)]}}

In [63]:
Molsys.reduced_ff_optimizable_values

{'all': {'CustomNonbondedForce': [array([[0.3234254759161587, 0.4240201476359742],
          [0.2157630343060931, 2.459952558318135],
          [0.32362767388845814, 0.5221353272207353]], dtype=object),
   array([[0.33580341155994586, 0.3447495018373475]], dtype=object)],
  'NonbondedForce': [array([[-0.4262896137364945],
          [1.1146153461471149],
          [-0.9103327452445908]], dtype=object)]},
 'mol1': {'NonbondedForce': [array([[-0.4262896137364945, 0.3234254759161587, 0.4240201476359742],
          [1.1146153461471149, 0.2157630343060931, 2.459952558318135],
          [-0.9103327452445908, 0.32362767388845814, 0.5221353272207353]],
         dtype=object)]}}

check optimizer

In [64]:
Para.optimizer.scipy_optimization

 final_simplex: (array([[-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
         0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.92703201,
         1.5174607 ],
       [-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
         0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.92703201,
         1.5174607 ],
       [-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
         0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.92703201,
         1.5174607 ],
       [-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
         0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.92703201,
         1.5174607 ],
       [-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
         0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.92703201,
         1.5174607 ],
       [-0.05667986, -0.00840798, -0.01379741,  6.18760802, -0.05311327,
         0.04423202, -0.04961809, -0.05185146,  0.4448418 ,  0.9270320

In [64]:
Molsys.openmm_systems['all'].nbfix

array([[0.3467687252569493, 0.08203582771612483, 'OG2P1', 'SOD'],
       [0.3467687252569493, 0.08203582771612483, 'SOD', 'OG2P1']],
      dtype=object)